# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Michelle Trigo  |
| **Fecha**      | 04/09/25  |
| **Expediente** | 745567  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Advertising.csv")
matriz = df.to_numpy()

In [3]:
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


### Sin penalización

In [4]:
matriz[:, 0] = 1
X = matriz[:, :-1]
y = matriz[:, 4].reshape(-1, 1)

In [5]:
ols = sm.OLS(y, X)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Sat, 06 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        01:34:07   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### L2

In [6]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
X=X[:, 1:]

In [8]:
ridge = Ridge(alpha=1)  
ridge.fit(X, y)
y_pred_ridge = ridge.predict(X)

r2 = r2_score(y, y_pred_ridge)

print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2",r2)

Intercepto [2.93896746] Coeficientes [[ 0.04576464  0.1885251  -0.00103629]] R2 0.8972106380074802


### L1

In [10]:
lasso = Lasso(alpha=1)  
lasso.fit(X, y)
y_pred_lasso = lasso.predict(X)

r2 = r2_score(y, y_pred_lasso)

print("Intercepto",lasso.intercept_,"Coeficientes",lasso.coef_, "R2",r2)

Intercepto [3.04021558] Coeficientes [0.04566142 0.1834644  0.        ] R2 0.8970235728389689


### Comparación

En la regresión sin penalización (OLS) se ve que las variables TV y Radio sí influyen en las ventas, mientras que Newspaper no tiene efecto importante, ya que su valor estadístico (p-value) es muy alto. El modelo explica cerca del 90% de las ventas (R² = 0.897).

Con la regresión Ridge (L2), los resultados son casi iguales a los de OLS. Los coeficientes cambian un poquito, pero el modelo sigue con el mismo nivel de precisión (R² = 0.8972). Aquí la variable Newspaper se mantiene en el modelo aunque no aporte mucho.

En la regresión Lasso (L1) los valores de TV y Radio se mantienen casi iguales, pero el coeficiente de Newspaper se reduce a cero. Esto muestra que Lasso tiende a eliminar variables que no son útiles, sin afectar el poder de predicción, que sigue siendo el mismo (R² = 0.8972).